# PART 1: SCANNING OMR SHEETS

## FUNCTION TO DRAW RECTANGLES

In [2]:
import cv2
import numpy as np

def drawRec(biggestNew, mainFrame):
        cv2.line(mainFrame, (biggestNew[0][0][0], biggestNew[0][0][1]), (biggestNew[1][0][0], biggestNew[1][0][1]), (0, 255, 0), 10)
        cv2.line(mainFrame, (biggestNew[0][0][0], biggestNew[0][0][1]), (biggestNew[2][0][0], biggestNew[2][0][1]), (0, 255, 0), 10)
        cv2.line(mainFrame, (biggestNew[3][0][0], biggestNew[3][0][1]), (biggestNew[2][0][0], biggestNew[2][0][1]), (0, 255, 0), 10)
        cv2.line(mainFrame, (biggestNew[3][0][0], biggestNew[3][0][1]), (biggestNew[1][0][0], biggestNew[1][0][1]), (0, 255, 0), 10)

## IMAGE PREPROCESSING

In [3]:
#initializing
img = cv2.imread('blanksheet 5.jpg') #read image
img = cv2.resize(img, (int(480*2), int(640*2)))
w, h = 480, 640
imgWarp = img.copy()

In [4]:
#preprocessing
GrayImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)                #converting to grayscale
BlurredFrame = cv2.GaussianBlur(GrayImg, (5, 5), 1)           #applying gaussian blur to smooth the image over
CannyFrame = cv2.Canny(BlurredFrame, 190, 190)                 #the smoothened picture's boundaries are detected

contours, _ = cv2.findContours(CannyFrame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#only detects external contour i.e edges and fixes the 4 vertices of the paper

ContourFrame = img.copy() #creating a copy
ContourFrame = cv2.drawContours(ContourFrame, contours, -1, (255, 0, 255), 4) #contour drawn in magenta colour of 4 pixel thickness
CornerFrame = img.copy()

In [5]:
#getting the biggest contour
maxArea = 0 #initializing the value to 0
biggest = [] #making an empty list
for i in contours :
    area = cv2.contourArea(i)  #calculating area enclosed inside the contour
    if area > 500 :
        peri = cv2.arcLength(i, True)
        edges = cv2.approxPolyDP(i, 0.02*peri, True)  #approximating the perimeter and the contour
        if area > maxArea and len(edges) == 4 :
            biggest = edges
            maxArea = area

if len(biggest) != 0 : #if the list is not empty
    
    biggest = biggest.reshape((4, 2))
    biggestNew = np.zeros((4, 1, 2), dtype= np.int32) #array to store coordinates
    add = biggest.sum(1)
    
    biggestNew[0] = biggest[np.argmin(add)]
    biggestNew[3] = biggest[np.argmax(add)]
    dif = np.diff(biggest, axis = 1)
    biggestNew[1] = biggest[np.argmin(dif)]
    biggestNew[2] = biggest[np.argmax(dif)]
    drawRec(biggestNew, CornerFrame)           #find coords of each corner and draw the rectangle
    
    CornerFrame = cv2.drawContours(CornerFrame, biggestNew, -1, (255, 0, 255), 25)
    pts1 = np.float32(biggestNew)                           #gets the detected rectangle's corners
    pts2 = np.float32([[0, 0], [w, 0], [0, h], [w, h]])     #coordinates of new perspective
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imgWarp = cv2.warpPerspective(img, matrix, (w, h))      #flattens the image

## SHOWING THE STAGES OF TRANSFORMATION

In [6]:
#saving

success = cv2.imwrite("StudentName.jpg", imgWarp)

if success:
    print("Success: Image saved successfully.")
else:
    print("ERROR: Save Failed. Please Check for errors in the program or the input image!")

Success: Image saved successfully.


In [7]:
#resizing
img = cv2.resize(img, (480, 640))
GrayImg = cv2.resize(GrayImg, (480, 640))
BlurredFrame = cv2.resize(BlurredFrame, (480, 640))
CannyFrame = cv2.resize(CannyFrame, (480, 640))
ContourFrame = cv2.resize(ContourFrame, (480, 640))
CornerFrame = cv2.resize(CornerFrame, (480, 640))

In [8]:
images = [img, GrayImg, BlurredFrame, CannyFrame, CornerFrame, ContourFrame, imgWarp]
images_names = ['input', 'GrayImg', 'BlurredFrame', 'CannyFrame', 'CornerFrame', 'ContourFrame', 'output']
x = 0
for i in images:
    cv2.imshow(images_names[x],i)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    x+=1

# PART 2: SHEET VALUATION

For this, we have to repeat the same procedure we had done in the document scanning code first.

In [9]:
import cv2
import numpy as np
import imutils

def drawRec(biggestNew, mainFrame):
        cv2.line(mainFrame, (biggestNew[0][0][0], biggestNew[0][0][1]), (biggestNew[1][0][0], biggestNew[1][0][1]), (0, 255, 0), 20)
        cv2.line(mainFrame, (biggestNew[0][0][0], biggestNew[0][0][1]), (biggestNew[2][0][0], biggestNew[2][0][1]), (0, 255, 0), 20)
        cv2.line(mainFrame, (biggestNew[3][0][0], biggestNew[3][0][1]), (biggestNew[2][0][0], biggestNew[2][0][1]), (0, 255, 0), 20)
        cv2.line(mainFrame, (biggestNew[3][0][0], biggestNew[3][0][1]), (biggestNew[1][0][0], biggestNew[1][0][1]), (0, 255, 0), 20)

In [10]:
Model_Answers = np.array([       #The actual answers are stored in this numpy array
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 0]])

# Getting the flattened image

In [11]:
img = cv2.imread('StudentName.jpg')
img = cv2.resize(img, (int(480*2), int(640*2)))
w, h = 480, 640
imgWarp = img.copy()

In [12]:
#preprocessing
GrayImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
BlurredFrame = cv2.GaussianBlur(GrayImg, (5, 5), 1)
CannyFrame = cv2.Canny(BlurredFrame, 190, 190)

#drawing contours
contours, _ = cv2.findContours(CannyFrame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
ContourFrame = img.copy()
ContourFrame = cv2.drawContours(ContourFrame, contours, -1, (255, 0, 255), 4)
CornerFrame = img.copy()

In [13]:
#getting the biggest contour i.e detecting the OMR field
maxArea = 0
biggest = []
for i in contours :
    area = cv2.contourArea(i)
    if area > 500 :
        peri = cv2.arcLength(i, True)
        edges = cv2.approxPolyDP(i, 0.02*peri, True)
        if area > maxArea and len(edges) == 4 :
            biggest = edges
            maxArea = area

if len(biggest) != 0 :
    biggest = biggest.reshape((4, 2))
    biggestNew = np.zeros((4, 1, 2), dtype= np.int32)
    add = biggest.sum(1)
    biggestNew[0] = biggest[np.argmin(add)]
    biggestNew[3] = biggest[np.argmax(add)]
    dif = np.diff(biggest, axis = 1)
    biggestNew[1] = biggest[np.argmin(dif)]
    biggestNew[2] = biggest[np.argmax(dif)]
    drawRec(biggestNew, CornerFrame)
    CornerFrame = cv2.drawContours(CornerFrame, biggestNew, -1, (255, 0, 255), 25)
    pts1 = np.float32(biggestNew)
    pts2 = np.float32([[0, 0], [w, 0], [0, h], [w, h]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imgWarp = cv2.warpPerspective(img, matrix, (w, h))

In [14]:
#resizing
img = cv2.resize(img, (480, 640))
GrayImg = cv2.resize(GrayImg, (480, 640))
BlurredFrame = cv2.resize(BlurredFrame, (480, 640))
CannyFrame = cv2.resize(CannyFrame, (480, 640))
ContourFrame = cv2.resize(ContourFrame, (480, 640))
CornerFrame = cv2.resize(CornerFrame, (480, 640))

In [15]:
imgWarpGray = cv2.cvtColor(imgWarp, cv2.COLOR_BGR2GRAY)                #changing colour scheme to grayscale
img_thresh = cv2.threshold(imgWarp,140,255, cv2.THRESH_BINARY_INV)[1]  #performing binary inverse on the pixels
img_thresh_cropped = img_thresh[10:-10,10:-10]                         #removing border artifacts

#cv2.imshow("img_thresh", img_thresh_cropped)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

# SPLITTING THE IMAGE INTO SMALLER PARTS

In [16]:
# Calculate the height of each slice
slice_height = img_thresh_cropped.shape[0] // 8

# Split the image into 8 horizontal slices
horizontal_slices = [img_thresh_cropped[i * slice_height: (i + 1) * slice_height, :] for i in range(8)]

# Display or process the horizontal slices as needed
for i, slice_img in enumerate(horizontal_slices):
    cv2.imshow(f"Slice {i}", slice_img)
    cv2.waitKey(2000)

cv2.destroyAllWindows()

In [17]:
# Initialize a list to store the vertical slices
vertical_slices = []

for horizontal_slice in horizontal_slices:
    # Calculate the width of each vertical slice
    slice_width = horizontal_slice.shape[1] // 4

    # Split the horizontal slice into 4 vertical slices
    for j in range(4):
        vertical_slice = horizontal_slice[:, j * slice_width: (j + 1) * slice_width]
        vertical_slices.append(vertical_slice)
        
        # Display or process the vertical slices as needed
        cv2.imshow(f"Vertical Slice {j} (from Horizontal Slice {i})", vertical_slice)
        cv2.waitKey(2000)

cv2.destroyAllWindows()

In [18]:
import cv2
import numpy as np

# Define the number of rows and columns
num_rows, num_cols = 8, 4

# Calculate the height and width of each slice
slice_height = img_thresh_cropped.shape[0] // num_rows
slice_width = img_thresh_cropped.shape[1] // num_cols

# Initialize an empty numpy array to store white pixel counts
white_counts = np.zeros((num_rows, num_cols), dtype=int)

# Iterate through each slice
for row in range(num_rows):
    for col in range(num_cols):
        # Extract the slice
        slice_img = img_thresh_cropped[row * slice_height: (row + 1) * slice_height,
                          col * slice_width: (col + 1) * slice_width]

        # Count white pixels (assuming white is represented by intensity 255)
        white_counts[row][col] = np.sum(slice_img == 255)

# Print the white pixel counts for each slice
print("White Pixel Counts for Each Slice:")
white_counts

White Pixel Counts for Each Slice:


array([[3135,  468,  306,  459],
       [ 285, 2646,  300,  405],
       [ 273,  126,  294,  411],
       [ 285,  435, 1590,  414],
       [ 267,  432,  294, 3315],
       [ 819,  417,  282,  423],
       [ 285,  438, 3186,  402],
       [ 309,  444,  315, 3678]])

Since the image was subjected to grayscale and then binary inverse, the black pixels have been converted to white pixels and greater concentration of white pixels shall indicate the answer marked by the student.

In [19]:
white_counts.shape

(8, 4)

In [20]:
answers = white_counts
for i in range(8):
    for j in range(4):
        if answers[i][j] < max(answers[i]):  #if maximum value is greater than current velue, store 0 in the cell
            answers[i][j] = 0

for i in range(8):
    for j in range(4):
        if answers[i][j] !=0:
            answers[i][j] = 1  #if the value stored in the cell is not 0 then make it 1
answers = np.array(answers)
answers                       #this array displays the answers marked by the student

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

In [21]:
Model_Answers #display model answers

array([[0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0]])

In [22]:
grade = 0.00
mistakes = []
for i in range(8):
    if list(answers[i]) == list(Model_Answers[i]):
        grade += 5.00 #add in 5 marks for every correct answer
    else:
        grade -= 0.5 #deduct 1/2 marks for every wrong answer
        mistakes.append(i+1)
print('TOTAL GRADES SCORED = ',grade)
print('MISTAKEN QUESTIONS:\t')
for i in mistakes:
    print("Q. ",i)

TOTAL GRADES SCORED =  18.0
MISTAKEN QUESTIONS:	
Q.  1
Q.  3
Q.  4
Q.  8
